In [ ]:
!pip install mlflow dagshub optuna imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [ ]:
import dagshub
import mlflow
dagshub.init(repo_owner='MihirPrajapati04', repo_name='yt-comment-sentiment-analysis', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/MihirPrajapati04/yt-comment-sentiment-analysis.mlflow")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=b0f32a56-0ef5-43db-98bb-195027dcac75&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=05d6d143d750a2afc02eca8419dc28a4d4ff915f1c6ed83e6652cb70168cced0




Output()

Accessing as MihirPrajapati04

Initialized MLflow to track repo "MihirPrajapati04/yt-comment-sentiment-analysis"

Repository MihirPrajapati04/yt-comment-sentiment-analysis initialized!

In [ ]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

2025/03/29 11:25:06 INFO mlflow.tracking.fluent: Experiment with name 'ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2c18439aef6f4d77be36aad757b9cb04', creation_time=1743247506507, experiment_id='4', last_update_time=1743247506507, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


In [ ]:
df = pd.read_csv('/content/reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [ ]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for KNN

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for KNN
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)  # Tuning the number of neighbors
    p = trial.suggest_categorical('p', [1, 2])  # Tuning the distance metric (1 for Manhattan, 2 for Euclidean)

    # KNeighborsClassifier setup
    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for KNN, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])

    # Log the best model with MLflow, passing the algo_name as "KNN"
    log_mlflow("KNN", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for KNN
run_optuna_experiment()


[I 2025-03-29 11:25:38,164] A new study created in memory with name: no-name-3a279632-e585-4f1a-a0e4-3eb81c786a47
[I 2025-03-29 11:25:47,138] Trial 0 finished with value: 0.5739801310505178 and parameters: {'n_neighbors': 6, 'p': 2}. Best is trial 0 with value: 0.5739801310505178.
[I 2025-03-29 11:25:56,256] Trial 1 finished with value: 0.5444937645318114 and parameters: {'n_neighbors': 19, 'p': 2}. Best is trial 0 with value: 0.5739801310505178.
[I 2025-03-29 11:26:05,340] Trial 2 finished with value: 0.5739801310505178 and parameters: {'n_neighbors': 6, 'p': 2}. Best is trial 0 with value: 0.5739801310505178.
[I 2025-03-29 11:26:31,973] Trial 3 finished with value: 0.3865990276897062 and parameters: {'n_neighbors': 13, 'p': 1}. Best is trial 0 with value: 0.5739801310505178.
[I 2025-03-29 11:26:40,748] Trial 4 finished with value: 0.5739801310505178 and parameters: {'n_neighbors': 6, 'p': 2}. Best is trial 0 with value: 0.5739801310505178.
[I 2025-03-29 11:26:49,831] Trial 5 finished

🏃 View run KNN_SMOTE_TFIDF_Trigrams at: https://dagshub.com/MihirPrajapati04/yt-comment-sentiment-analysis.mlflow/#/experiments/4/runs/c6b48b5d62e14821b0b48250c6fcd061
🧪 View experiment at: https://dagshub.com/MihirPrajapati04/yt-comment-sentiment-analysis.mlflow/#/experiments/4
